In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [18]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://job-boards.greenhouse.io/gitlab/jobs/8231973002") #active job posting
pagedata = loader.load().pop().page_content
print (pagedata)

Job Application for Associate Support Engineer (AMER - PST / MST) at GitLabAssociate Support Engineer (AMER - PST / MST)Remote, AmericasApplyGitLab is an open-core software company that develops the most comprehensive AI-powered DevSecOps Platform, used by more than 100,000 organizations. Our mission is to enable everyone to contribute to and co-create the software that powers our world. When everyone can contribute, consumers become contributors, significantly accelerating human progress. Our platform unites teams and organizations, breaking down barriers and redefining what's possible in software development. Thanks to products like Duo Enterprise and Duo Agent Platform, customers get AI benefits at every stage of the SDLC. 
The same principles built into our products are reflected in how our team works: we embrace AI as a core productivity multiplier, with all team members expected to incorporate AI into their daily workflows to drive efficiency, innovation, and impact. GitLab is wh

In [29]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
     ###SCRAPED TEXT FROM WEBSITE:
    {pagedata}

    ###INSTRUCTION:
    The scraped text is from the careers page of a website.
    Your job: extract the job posting(s) and return a JSON format single object  containing following keys.
    Each object must contain exactly these keys: `role`, `experience`, `skills`, and  `description`.
    - If a field is not present in the scraped text, return an empty string for "experience" and description, or an empty array for "skills".

    ONLY RETURN VALID JSON (no PREAMBLE, no explanation, no extra fields). No punctuation or character before or after Json 
    

    ###VALID JSON(NO PREAMBLE):
    """

)

chain_extract = prompt_extract| llm
res = chain_extract.invoke(input={'pagedata': pagedata})
print(res.content)

{"role": "Associate Support Engineer", "experience": "", "skills": ["Linux systems knowledge", "scripting languages", "Git", "communication"], "description": "Support a mix of Self-managed and GitLab.com (SaaS) customers and resolve their issues via Zendesk tickets, merge requests, email and video conferencing"}


In [30]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


{'role': 'Associate Support Engineer',
 'experience': '',
 'skills': ['Linux systems knowledge',
  'scripting languages',
  'Git',
  'communication'],
 'description': 'Support a mix of Self-managed and GitLab.com (SaaS) customers and resolve their issues via Zendesk tickets, merge requests, email and video conferencing'}

In [31]:
type(json_res)

dict

In [33]:
import pandas as pd

df= pd.read_csv("my_portfolio.csv")

In [56]:
import chromadb
import uuid
from chromadb.utils import embedding_functions

ef = embedding_functions.DefaultEmbeddingFunction()
client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name="portfolio", embedding_function=ef)

for _, row in df.iterrows():
    collection.add(
        documents=[row["Techstack"]],
        metadatas=[{"links": row["Links"]}],
        ids=[str(uuid.uuid4())]
    )

links = collection.query(
    query_texts=["Linux", "Git","Scripting","Python"],
    n_results=2
).get("metadatas")

print(links)


[[{'links': 'https://example.com/java-portfolio'}, {'links': 'https://example.com/java-portfolio'}], [{'links': 'https://example.com/devops-portfolio'}, {'links': 'https://example.com/devops-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/ml-python-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/ml-python-portfolio'}]]


In [54]:
job = json_res
job['skills']

['Linux systems knowledge', 'scripting languages', 'Git', 'communication']

In [57]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at RenderiQ. RenderiQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of RenderiQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase RenderiQ's portfolio: {link_list}
        Remember you are Mohan, BDE at RenderiQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Expert Support Engineering Solutions for Seamless Customer Experience

Dear Hiring Manager,

I came across the Associate Support Engineer role at your esteemed organization and was impressed by the job description. As a Business Development Executive at RenderiQ, an AI & Software Consulting company, I believe our expertise can help fulfill your support engineering needs.

At RenderiQ, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team possesses extensive knowledge of Linux systems, scripting languages, and Git, which aligns perfectly with the requirements of the Associate Support Engineer role. We also excel in communication, ensuring seamless collaboration with your customers via Zendesk tickets, merge requests, email, and video conferencing.

Our portfolio showcases our capabilities in DevOps, which is highly relevant to the role. You can ex